In [16]:
import numpy as np
from collections import defaultdict

In [17]:
def mean_average_precision(actual_interactions, predicted_interactions, k):
    map_scores = []
    for user_id, actual_items in actual_interactions.items():
        if user_id in predicted_interactions:
            predicted_items = [item[0] for item in predicted_interactions[user_id]][:k]
            map_score = average_precision(actual_items, predicted_items, k)
            map_scores.append(map_score)
    return np.mean(map_scores)

In [18]:
def average_precision(actual_items, predicted_items, k):
    num_hits = 0
    precision_at_k = []
    for i, item in enumerate(predicted_items):
        if item in actual_items:
            num_hits += 1
            precision_at_k.append(num_hits / (i + 1))
    if not precision_at_k:
        return 0
    return np.sum(precision_at_k) / min(len(actual_items), k)

In [1]:
def discounted_cumulative_gain(actual_items, predicted_items, k):
    dcg = 0
    for i, item in enumerate(predicted_items):
        if item in actual_items:
            dcg += 1 / np.log2(i + 2)
    return dcg

def ideal_discounted_cumulative_gain(actual_items, k):
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(actual_items), k)))
    return idcg

In [19]:
def normalized_discounted_cumulative_gain(actual_interactions, predicted_interactions, k):
    ndcg_scores = []
    for user_id, actual_items in actual_interactions.items():
        if user_id in predicted_interactions:
            predicted_items = [item[0] for item in predicted_interactions[user_id]][:k]
            ndcg_score = discounted_cumulative_gain(actual_items, predicted_items, k)
            idcg_score = ideal_discounted_cumulative_gain(actual_items, k)
            if idcg_score == 0:
                ndcg_scores.append(0)
            else:
                ndcg_scores.append(ndcg_score / idcg_score)
    return np.mean(ndcg_scores)

In [21]:
def coverage(predicted_interactions, catalog_size):
    unique_items = set(item for sublist in predicted_interactions.values() for item, _ in sublist)
    return len(unique_items) / catalog_size

def novelty(predicted_interactions, item_popularity):
    novelty_scores = []
    for user_id, predictions in predicted_interactions.items():
        user_novelty = 0
        for item_id, _ in predictions:
            user_novelty += item_popularity.get(item_id, 0)
        novelty_scores.append(user_novelty / len(predictions))
    return np.mean(novelty_scores)

In [22]:
def mean_reciprocal_rank(actual_interactions, predicted_interactions):
    rr_scores = []
    for user_id, actual_items in actual_interactions.items():
        if user_id in predicted_interactions:
            predicted_items = [item[0] for item in predicted_interactions[user_id]]
            rr_score = reciprocal_rank(actual_items, predicted_items)
            rr_scores.append(rr_score)
    return np.mean(rr_scores)

def reciprocal_rank(actual_items, predicted_items):
    for i, item in enumerate(predicted_items):
        if item in actual_items:
            return 1 / (i + 1)
    return 0

In [27]:
# Fake data 
num_users = 100
num_items = 500
num_recommendations = 10
catalog_size = 1000

In [31]:

actual_interactions = {}
for user_id in range(num_users):
    num_interactions = np.random.randint(1, 20)
    items = np.random.choice(num_items, size=num_interactions, replace=False)
    actual_interactions[user_id] = list(items)


predicted_interactions = defaultdict(list)
for user_id in range(num_users):
    recommendations = [(np.random.randint(num_items), np.random.rand()) for _ in range(num_recommendations)]
    predicted_interactions[user_id] = recommendations

# random item popularity scores
item_popularity = {item_id: np.random.rand() for item_id in range(num_items)}


In [25]:
# Compute performance metrics
k = 10
map_score = mean_average_precision(actual_interactions, predicted_interactions, k)
ndcg_score = normalized_discounted_cumulative_gain(actual_interactions, predicted_interactions, k)
cov_score = coverage(predicted_interactions, catalog_size)
novelty_score = novelty(predicted_interactions, item_popularity)
mrr_score = mean_reciprocal_rank(actual_interactions, predicted_interactions)

In [26]:
print("Mean Average Precision (MAP):", map_score)
print("Normalized Discounted Cumulative Gain (NDCG):", ndcg_score)
print("Coverage:", cov_score)
print("Novelty:", novelty_score)
print("Mean Reciprocal Rank (MRR):", mrr_score)

Mean Average Precision (MAP): 0.007836224489795918
Normalized Discounted Cumulative Gain (NDCG): 0.02209089756520599
Coverage: 0.431
Novelty: 0.5103124443957959
Mean Reciprocal Rank (MRR): 0.04314682539682539
